In [43]:
import pandas as pd
import lou.NL as lou

In [ ]:
def prep_df_to_lou(df, normalize=True, train_size=0):
    '''
    Function that converts a pandas df to numpy list ready to be processed by a lou network

    You need to prepare the df first, but don't worry about normalization, the function can do it.

    It assumes last collumn is prediction
    '''
    def normalizer(data, column):
        min_val = data[column].min()
        max_val = data[column].max()
        data[column] = (data[column] - min_val) / (max_val - min_val)



    if normalize :
        for col in df:
            normalizer(df,col)

    res = []

    df_train = df.head(df.shape[0]-train_size)

    df_test = df.tail(100)
    
    train_data = []
    for index, row in df_train.iterrows() :
        train_data.append((row.iloc[:-1].to_numpy().reshape(-1,1), row.iloc[-1:].to_numpy().reshape(1,1)))

    if train_size > 0 :
        test_data = []
        for index, row in df_test.iterrows() :
            test_data.append((row.iloc[:-1].to_numpy().reshape(-1,1), row.iloc[-1:].to_numpy().reshape(1,1))) 
        res.append(test_data)
    
    return res
    

    

SyntaxError: can't use starred expression here (825340057.py, line 36)

In [ ]:
df  = pd.read_csv('playground-series-s5e5/train.csv')

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0
...,...,...,...,...,...,...,...,...,...
749995,749995,male,28,193.0,97.0,30.0,114.0,40.9,230.0
749996,749996,female,64,165.0,63.0,18.0,92.0,40.5,96.0
749997,749997,male,60,162.0,67.0,29.0,113.0,40.9,221.0
749998,749998,male,45,182.0,91.0,17.0,102.0,40.3,109.0


In [28]:
df = df.drop(columns=['id'])
df['Sex']=df['Sex'].map({'male':0, 'female':1})

In [30]:
train_data, test_data = prep_df_to_lou(df)
print(train_data[0])

(array([[0.        ],
       [0.27118644],
       [0.65625   ],
       [0.47916667],
       [0.86206897],
       [0.55737705],
       [0.88636364]]), array([[0.47603834]]))


In [44]:
Network = lou.NeuralNetwork([7,10,5,5,1])

In [46]:
print(f'cost before : {Network.meancost(test_data)}')
Network.train(train_data, alpha=0.1, tries=1000, verbose=10)
print(f'cost after : {Network.meancost(test_data)}')
Network.export('network')

cost before : 0.2606834553689045
Loop number 0
Loop number 10
Loop number 20
Loop number 30
Loop number 40
Loop number 50


KeyboardInterrupt: 

In [63]:
test_df = pd.read_csv('playground-series-s5e5/test.csv')
id = test_df['id']
data = test_df.drop(columns=['id'])
data['Sex'] = data['Sex'].map({'male':0, 'female':1})

In [75]:
data_np = []
for i, row in data.iterrows():
    data_np.append(row.to_numpy().reshape(7,1))

In [77]:
# get the min and max to normalize back :
train_data = pd.read_csv('playground-series-s5e5/train.csv')
min = train_data['Calories'].min()
max = train_data['Calories'].max()

In [81]:
res = []
for line in data_np:
    tmp = Network.frontprop(line)[-1][0][0]
    res.append(tmp*(max-min)+min)

res_df = pd.DataFrame({"id" : id, "Calories" : res})

In [83]:
res_df.to_csv('submission.csv', index=False)